<table>
  <tr>
    <th>
    Wydział Informatyki, Katedra SIiSK </br> PS Bezpieczeństwa Sieci            Komputerowych
    </th>
    <th>
    Data: 16.03.2021
    </th>
  </tr>
  <tr>
    <td>
    Ćwiczenie nr 3</br>Temat: Implementacja podstawowych modułów kryptograficznych </br>Grupa nr 2</br>Artur Porowski
    </td>
    <td>
    Prowadzący:  mgr inż. Dariusz Jankowski</br>Ocena:
    </td>
  </tr>
<table>

Zadanie sprawdzone na zajęciach 23.03.2021

In [ ]:
import doctest
import math
import string

# Zadanie 1
## Treść
Zaimplementuj algorytm kodujący i dekodujący z wykorzystaniem szyfru prostego przestawiania „rail fence" dla *k = n*. Skorzystaj z przykładu 1 (1 punkt).
## Rozwiązanie

In [ ]:
def encodeRailFence(word, n):
    """Return encoded word in Rail Fence algorithm with depth of n.
    
    Base case #1:
    >>> encodeRailFence('CRYPTOGRAPHY', 3)
    'CTARPORPYYGH'

    Base case #2:
    >>> encodeRailFence('BSKTRZEBAZDAC', 6)
    'BDSZAKACTBREZ'

    When depth n is =1, encrypted word should be the same as passed word:
    >>> encodeRailFence('CRYPTOGRAPHY', 1)
    'CRYPTOGRAPHY'

    When depth n is >= len(word), then encrypted word should be the same as passed word:
    >>> encodeRailFence('CRYPTOGRAPHY', 12)
    'CRYPTOGRAPHY'

    When depth n is <=0, there should be error thrown:
    >>> encodeRailFence('CRYPTOGRAPHY', 0)
    Traceback (most recent call last):
        ...
    ValueError: n must be greater than 0

    When depth n isn't an integer, there should be error thrown:
    >>> encodeRailFence('CRYPTOGRAPHY', 1.2)
    Traceback (most recent call last):
        ...
    ValueError: n must be exact integer
    """
    if(n<=0):
        raise ValueError("n must be greater than 0")
    if(math.floor(n) != n):
        raise ValueError("n must be exact integer")
    if(n==1):
        return word

    parts = []
    for i in range(n):
        parts.append("")

    d = 0 # Current depth
    isGoingDown = True # Is going down the 'fence' in the algorithm
    for i in range(len(word)):
      if(d==0):
        isGoingDown = True
      if(d==n-1):
        isGoingDown = False
      
      parts[d] += word[i]
      d += 1 if isGoingDown else -1
        
    output = ""
    for word_part in parts:
      output += word_part # Concatenate the output from parts
    
    return output

def decodeRailFence(word, n):
    """Return decoded word from Rail Fence algorithm with depth of n.
    
    Base case #1:
    >>> decodeRailFence('CTARPORPYYGH', 3)
    'CRYPTOGRAPHY'

    Base case #2:
    >>> decodeRailFence('BDSZAKACTBREZ', 6)
    'BSKTRZEBAZDAC'

    When depth n is =1, encrypted word should be the same as passed word:
    >>> decodeRailFence('CRYPTOGRAPHY', 1)
    'CRYPTOGRAPHY'

    When depth n is >= len(word), then encrypted word should be the same as passed word:
    >>> decodeRailFence('CRYPTOGRAPHY', 12)
    'CRYPTOGRAPHY'

    When depth n is <=0, there should be error thrown:
    >>> decodeRailFence('CRYPTOGRAPHY', 0)
    Traceback (most recent call last):
        ...
    ValueError: n must be greater than 0

    When depth n isn't an integer, there should be error thrown:
    >>> encodeRailFence('CRYPTOGRAPHY', 1.2)
    Traceback (most recent call last):
        ...
    ValueError: n must be exact integer
    """
    if(n<=0):
        raise ValueError("n must be greater than 0")
    if(math.floor(n) != n):
        raise ValueError("n must be exact integer")
    if(n==1):
        return word

    wordLength = len(word)

    parts = []
    dots = []
    for i in range(n):
        parts.append("")
        dots.append("")

    d = 0 # Current depth
    isGoingDown = True # Is going down the 'fence' in the algorithm
    for i in range(len(word)):
      if(d==0):
        isGoingDown = True
      if(d==n-1):
        isGoingDown = False
      
      dots[d] += "*" # Build "*" filled parts to know how many characters are in each depth layer
      d += 1 if isGoingDown else -1

    for i in range(len(dots)):
      word, parts[i] = word[len(dots[i]):], word[:len(dots[i])]

    output = ""
    d = 0 # Current depth
    isGoingDown = True # Is goin down the 'fence' in the algorithm
    for i in range(wordLength):
      if(d==0):
        isGoingDown = True
      if(d==n-1):
        isGoingDown = False

      parts[d], res = parts[d][1:], parts[d][0] # Cut first letter from the part at the current depth
      output += res

      d += 1 if isGoingDown else -1
      
    return output

# If every test is good it wont print anything
doctest.run_docstring_examples(encodeRailFence, globals(), name="Rail Fence Encoder")
doctest.run_docstring_examples(decodeRailFence, globals(), name="Rail Fence Decoder")

## Opis algorytmu
### Szyfrowanie
Przechodząc przez słowo wejściowe inkrementuję lub dekrementuję index głębokości, który służy do wybrania warstwy do której zostaje dopisana litera. Decyzja dotycząca in-/de-krementacji podejmowana jest na podstawie kierunku poruszania się "w dół" lub "w górę" płotka. Kierunek jest odwracany, gdy index głębokości jest równy 0 lub maksymalnej głębkości.

### Deszyfrowanie
Najpierw buduję "scaffold" warstw wypełniony "*", aby wiedzieć ile znaków powinno się znajdować w każdej warstwie przy ustalonej głębokości maksymalnej.
Następnie przepisuję do nich odpowiednie części zakodowanego słowa.
Na zakończenie przechodząc przez całe słowo jak w przypadku szyfrowania, biorę z odpowiedniej warstwy pierwszą literę, dopisuję ją do wyjścia i wyrzucam ją z tej części.

# Zadanie 2
## Treść
Zaimplementuj kryptosystem przestawieniowy bazujący na przykładzie 2a dla *d = 5* oraz klucza *key = 3-4-1-5-2* (1 punkt).
## Rozwiązanie

In [ ]:
def encodeMatrixCypherA(word):
    """Return encoded word in Matrix Cypher algorithm with length of block 5 and key=3-1-4-5-2.

    Base case #1:
    >>> encodeMatrixCypherA('CRYPTOGRAPHY')
    'YPCTRRAOPGHY'

    Base case #2:
    >>> encodeMatrixCypherA('CRYPTOGRAPHYOS')
    'YPCTRRAOPGOSHY'

    If word is empty it should return word:
    >>> encodeMatrixCypherA('')
    ''

    If word is shorter than key it still should encode the word:
    >>> encodeMatrixCypherA('CRY')
    'YCR'
    """
    word = str(word).replace(" ", "").upper()

    d=5
    key=[3,4,1,5,2]
    idx = 0
    parts = [""]

    # Split word in correct sized parts
    for letter in word:
      if(len(parts[idx]) == d):
        parts.append("")
        idx += 1
      parts[idx] += letter

    # Build output based on the key
    output = ""
    for part in parts:
      for idx in key:
        if (idx-1) in range(len(part)):
          output += part[idx-1]
    
    return output

def decodeMatrixCypherA(word):
    """Return decoded word from Matrix Cypher algorithm with length of block 5 and key=3-1-4-5-2.

    Base case #1:
    >>> decodeMatrixCypherA('YPCTRRAOPGHY')
    'CRYPTOGRAPHY'

    Base case #2:
    >>> decodeMatrixCypherA('YPCTRRAOPGOSHY')
    'CRYPTOGRAPHYOS'

    If word is empty it should return word:
    >>> decodeMatrixCypherA('')
    ''

    If word is shorter than key it still should decode the word:
    >>> decodeMatrixCypherA('YCR')
    'CRY'
    """
    word = str(word).replace(" ", "").upper()

    d=5
    key=[3,4,1,5,2]
    idx = 0
    parts = [""]

    # Split word in correct sized parts
    for letter in word:
      if(len(parts[idx]) == d):
        parts.append("")
        idx += 1
      parts[idx] += letter

    output = ""
    for part in parts:

      # Inverse permutation of the key after deleting indexes not in the length of the line
      tmp = list(filter(lambda x: x<=len(part), key))
      k = [0] * len(part)
      for idx in range(len(tmp)):
          k[tmp[idx]-1] = idx+1

      # Build output based on the key
      for idx in k:
        output += part[idx-1]
    
    return output

# If every test is good it wont print anything
doctest.run_docstring_examples(encodeMatrixCypherA, globals(), name="Matrix Cypher A Encoder")
doctest.run_docstring_examples(decodeMatrixCypherA, globals(), name="Matrix Cypher A Decoder")

## Opis algorytmu
### Szyfrowanie
Najpierw wczytuję słowo dzieląc je na części długości d.
Następnie buduję słowo zakodowane, dla każdej części wiedząc, że key >= długości części mogę poprostu korzystając z indeksów zapisanych w kluczu dodawać do wyjścia odpowiednie litery. Oczywiście obowiązkowym jest sprawdzenie czy indeks z klucza znajduje się w przedziale <0,dł_części).

### Deszyfrowanie
Algorytm deszyfrowania jest prawie identyczny do algorytmu szyfrowania. Różnicami są: odwrócenie klucza (algorytm odwórcenia permutacji) i jednocześnie odfiltrowanie klucza aby nie posiadał wartości większych niż długość części.

# Zadanie 3
## Treść
Zaimplementuj kryptosystem przestawieniowy bazujący na przykładzie 2b (1 punkt) oraz 2c (2 punkty) dla dowolnego klucza.
## Rozwiązanie

In [ ]:
def encodeMatrixCypherB(word, password):
    """Return encoded word in Matrix Cypher algorithm with key generated from password.

    Base case #1:
    >>> encodeMatrixCypherB('CRYPTOGRAPHY', 'CONVENIENCE')
    'CYPTRHGYOARP'

    Base case #2:
    >>> encodeMatrixCypherB('CRYPTOGRAPHYOS', 'CONVENIENCE')
    'CYPTRHGYSOAROP'

    If word is empty it should return word:
    >>> encodeMatrixCypherB('', 'CONVENIENCE')
    ''

    If password is empty it should return empty string:
    >>> encodeMatrixCypherB('CYPHER', '')
    ''

    If word is shorter than key it still should encode the word:
    >>> encodeMatrixCypherB('CRY', 'CONVENIENCE')
    'CYR'

    Password has to be only from english alphabet:
    >>> encodeMatrixCypherB('CRYPTOGRAPHY', 1)
    Traceback (most recent call last):
        ...
    ValueError: password can't contain a number
    """
    if(isinstance(password, int) or any(chr.isdigit() for chr in password)):
      raise ValueError("password can't contain a number")

    word = str(word).replace(" ", "").upper()

    d=len(password)
    key=[0]*d

    # Generating key based on passcode
    i = 1
    for ch in string.ascii_uppercase:
        if ch in password:
            indices = list(filter(lambda x: password[x]==ch, range(d)))
            for index in indices:
                key[index] = i
                i+=1

    idx = 0
    parts = [""]

    # Split word in correct sized parts
    for letter in word:
      if(len(parts[idx]) == d):
        parts.append("")
        idx += 1
      parts[idx] += letter 

    # Build output based on the key
    output = ""
    for i in range(1,d+1):
      idx = key.index(i)
      for part in parts:
        if idx in range(len(part)):
            output += part[idx] 
    
    return output

def decodeMatrixCypherB(word, password):
    """Return decoded word from Matrix Cypher algorithm with key generated from password.

    Base case #1:
    >>> decodeMatrixCypherB('CYPTRHGYOARP', 'CONVENIENCE')
    'CRYPTOGRAPHY'

    Base case #2:|
    >>> decodeMatrixCypherB('CYPTRHGYSOAROP', 'CONVENIENCE')
    'CRYPTOGRAPHYOS'

    If word is empty it should return word:
    >>> decodeMatrixCypherB('', 'CONVENIENCE')
    ''

    If password is empty it should return empty string:
    >>> encodeMatrixCypherB('CYPHER', '')
    ''

    If word is shorter than key it still should decode the word:
    >>> decodeMatrixCypherB('CYR', 'CONVENIENCE')
    'CRY'

    Password has to be only from english alphabet:
    >>> decodeMatrixCypherB('CRYPTOGRAPHY', 1)
    Traceback (most recent call last):
        ...
    ValueError: password can't contain a number
    """
    if(isinstance(password, int) or any(chr.isdigit() for chr in password)):
      raise ValueError("password can't contain a number")

    word = str(word).replace(" ", "").upper()

    d=len(password)
    key=[0]*d

    # Generating key based on passcode
    i = 1
    for ch in string.ascii_uppercase:
        if ch in password:
            indices = list(filter(lambda x: password[x]==ch, range(d)))
            for index in indices:
                key[index] = i
                i+=1

    idx = 0
    parts = [[]]

    # Prepare scaffold in correct sized parts
    for letter in word:
        if(len(parts[idx]) == d):
            parts.append([])
            idx += 1
        parts[idx] += "*"

    # Based on scaffold split letters in correct parts
    j = 0
    for i in range(1,d+1):
        idx = key.index(i)
        for n, part in enumerate(parts):
            if idx in range(len(part)):
                parts[n][idx] = word[j]
                j+=1

    # Build output from parts
    output = ""
    for part in parts:
        for ch in part:
            output += ch

    return output

# If every test is good it wont print anything
doctest.run_docstring_examples(encodeMatrixCypherB, globals(), name="Matrix Cypher B Encoder")
doctest.run_docstring_examples(decodeMatrixCypherB, globals(), name="Matrix Cypher B Decoder")

## Opis algorytmu
### Szyfrowanie
Pierwszą rzeczą jest zbudowanie klucza w oparciu o słowo kodowe, przechodzę po każdej literze alfabetu, jeśli znajduje się ono w słowie to na odpowiednim indeksie tabeli wprowadzam wartość inkrementowaną od 1 z każdym jej przypisaniem. Ważnym jest aby uwzględnić powtórzenia tej samej litery w haśle.  
Potem wczytuję słowo dzieląc je na części długości d - długość hasła.
Następnie buduję słowo zakodowane, dla każdego indeksu w haśle przechodzę przez części słowa i korzystując z indeksów wpisuje je do słowa zakodowanego. Służy to odwróceniu całych kolumn macierzy.

### Deszyfrowanie
Pierwszą rzeczą jest zbudowanie klucza w oparciu o słowo kodowe, przechodzę po każdej literze alfabetu, jeśli znajduje się ono w słowie to na odpowiednim indeksie tabeli wprowadzam wartość inkrementowaną od 1 z każdym jej przypisaniem. Ważnym jest aby uwzględnić powtórzenia tej samej litery w haśle.  
Potem buduję 'scaffold' aby uzyskać odpowiednie długości poszczególnych części hasła kodowanego. Następnie na podstawie klucza przechodzę przez jego indeksy i enumerując części wzorca uzupełniam go odpowiednimi literami z kolumn.

In [ ]:
def encodeMatrixCypherC(word, password):
    """Return encoded word in Matrix Cypher algorithm with key generated from password. 
    More irregular modification.

    Base case #1:
    >>> encodeMatrixCypherC('CRYPTOGRAPHY', 'CONVENIENCE')
    'CRYHOARPGPYT'

    Base case #2:
    >>> encodeMatrixCypherC('CRYPTOGRAPHYOS', 'CONVENIENCE')
    'CRYHOARPSGPYOT'

    If word is empty it should return word:
    >>> encodeMatrixCypherC('', 'CONVENIENCE')
    ''

    If password is empty it should return empty string:
    >>> encodeMatrixCypherB('CYPHER', '')
    ''

    If word is shorter than key it should return the word:
    >>> encodeMatrixCypherC('CRYPTO', 'CONVENIENCE')
    'CROPYT'

    Password has to be only from english alphabet:
    >>> encodeMatrixCypherC('CRYPTOGRAPHY', 1)
    Traceback (most recent call last):
        ...
    ValueError: password can't contain a number
    """
    if(isinstance(password, int) or any(chr.isdigit() for chr in password)):
      raise ValueError("password can't contain a number")

    word = str(word).replace(" ", "").upper()

    d=len(password)
    key=[0]*d

    # Generating key based on passcode
    i = 1
    for ch in string.ascii_uppercase:
        if ch in password:
            indices = list(filter(lambda x: password[x]==ch, range(d)))
            for index in indices:
                key[index] = i
                i+=1

    idx = 0
    parts = [""]

    # Split word in correct sized parts
    for letter in word:
      i = key.index(idx+1)
      if(len(parts[idx]) == i+1):
        parts.append("")
        idx += 1
      parts[idx] += letter 

    # Build output based on the key
    output = ""
    for i in range(1,d+1):
      idx = key.index(i)
      for part in parts:
        if idx in range(len(part)):
            output += part[idx] 
    
    return output

def decodeMatrixCypherC(word, password):
    """Return decoded word from Matrix Cypher algorithm with key generated from password.
    More irregular modification.

    Base case #1:
    >>> decodeMatrixCypherC('CRYHOARPGPYT', 'CONVENIENCE')
    'CRYPTOGRAPHY'

    Base case #2:
    >>> decodeMatrixCypherC('CRYHOARPSGPYOT', 'CONVENIENCE')
    'CRYPTOGRAPHYOS'

    If word is empty it should return word:
    >>> decodeMatrixCypherC('', 'CONVENIENCE')
    ''

    If password is empty it should return empty string:
    >>> encodeMatrixCypherB('CYPHER', '')
    ''

    If word is shorter than key it still should decode the word:
    >>> decodeMatrixCypherC('CROPYT', 'CONVENIENCE')
    'CRYPTO'

    Password has to be only from english alphabet:
    >>> encodeMatrixCypherC('CRYPTOGRAPHY', 1)
    Traceback (most recent call last):
        ...
    ValueError: password can't contain a number
    """
    if(isinstance(password, int) or any(chr.isdigit() for chr in password)):
      raise ValueError("password can't contain a number")

    word = str(word).replace(" ", "").upper()

    d=len(password)
    key=[0]*d

    # Generating key based on passcode
    i = 1
    for ch in string.ascii_uppercase:
        if ch in password:
            indices = list(filter(lambda x: password[x]==ch, range(d)))
            for index in indices:
                key[index] = i
                i+=1

    idx = 0
    parts = [[]]

    # Prepare scaffold in correct sized parts
    for letter in word:
        i = key.index(idx+1)
        if(len(parts[idx]) == i+1):
            parts.append([])
            idx += 1
        parts[idx] += "*"

    # Based on scaffold split letters in correct parts
    j = 0
    for i in range(1,d+1):
        idx = key.index(i)
        for n, part in enumerate(parts):
            if idx in range(len(part)):
                parts[n][idx] = word[j]
                j+=1

    # Build output from parts
    output = ""
    for part in parts:
        for ch in part:
            output += ch

    return output

# If every test is good it wont print anything
doctest.run_docstring_examples(encodeMatrixCypherC, globals(), name="Matrix Cypher C Encoder")
doctest.run_docstring_examples(decodeMatrixCypherC, globals(), name="Matrix Cypher C Decoder")

## Opis algorytmu
### Szyfrowanie
Pierwszą rzeczą jest zbudowanie klucza w oparciu o słowo kodowe, przechodzę po każdej literze alfabetu, jeśli znajduje się ono w słowie to na odpowiednim indeksie tabeli wprowadzam wartość inkrementowaną od 1 z każdym jej przypisaniem. Ważnym jest aby uwzględnić powtórzenia tej samej litery w haśle.  
Potem wczytuję słowo dzieląc je na części długości d - indeksu numeru rzędu w kluczu + 1.
Następnie buduję słowo zakodowane, dla każdego indeksu w haśle przechodzę przez części słowa i korzystując z indeksów wpisuje je do słowa zakodowanego. Służy to odwróceniu całych kolumn macierzy.

### Deszyfrowanie
Pierwszą rzeczą jest zbudowanie klucza w oparciu o słowo kodowe, przechodzę po każdej literze alfabetu, jeśli znajduje się ono w słowie to na odpowiednim indeksie tabeli wprowadzam wartość inkrementowaną od 1 z każdym jej przypisaniem. Ważnym jest aby uwzględnić powtórzenia tej samej litery w haśle.  
Potem buduję 'scaffold' aby uzyskać odpowiednie długości poszczególnych części hasła kodowanego. Następnie na podstawie klucza przechodzę przez jego indeksy i enumerując części wzorca uzupełniam go odpowiednimi literami z kolumn.

# Zadanie 4
## Treść
Zaimplementuj szyfr cezara bazujący na przykładzie 3b (1 punkt).
## Rozwiązanie

In [ ]:
def encodeCeasarModified(word, k0, k1):
    """Return encoded word with (a*k1+k0) mod n, for n=26

    Base case #1:
    >>> encodeCeasarModified('CRYPTOGRAPHY', 0, 3)
    'GZUTFQSZATVU'

    Base case #2:
    >>> encodeCeasarModified('CRYPTOGRAPHYOS', 5, 7)
    'TURGIZVUFGCRZB'

    If k0 and k1 aren't coprimes:
    >>> encodeCeasarModified("whatever", 4, 8)
    Traceback (most recent call last):
        ...
    ValueError: n and k1 must be coprimes

    If k1 is equal to 0:
    >>> encodeCeasarModified("whatever", 4, 0)
    Traceback (most recent call last):
        ...
    ValueError: k1 has to be greater than 0

    Word has to only consist of letters from english alphabet:
    >>> encodeCeasarModified('CRYP 223KKK', 1, 0)
    Traceback (most recent call last):
        ...
    ValueError: word can't contain a number
    """
    if(isinstance(word, int) or any(chr.isdigit() for chr in word)):
      raise ValueError("word can't contain a number")

    word = str(word).replace(" ", "").upper()

    n=26

    if(k1 <= 0):
      raise ValueError("k1 has to be greater than 0")

    if(math.gcd(n, k1) != 1):
      raise ValueError("n and k1 must be coprimes")

    abc = string.ascii_uppercase
    output = ""
    for letter in word:
      idx = abc.index(letter)
      outidx = (idx*k1+k0) % n
      output += abc[outidx]

    return output

def phi(n): #funckja tocjent Eulera
    v = 0
    for i in range(n):
      if(math.gcd(n, i) == 1):
        v += 1
    return v

def decodeCeasarModified(word, k0, k1):
    """Return decoded word with [c+(n-k0)]k1^ϕ(n)−1 mod n, for n=21

    Base case #1:
    >>> decodeCeasarModified('GZUTFQSZATVU', 0, 3)
    'CRYPTOGRAPHY'

    Base case #2:
    >>> decodeCeasarModified('TURGIZVUFGCRZB', 5, 7)
    'CRYPTOGRAPHYOS'

    If k0 and k1 aren't coprimes:
    >>> decodeCeasarModified("whatever", 4, 8)
    Traceback (most recent call last):
        ...
    ValueError: n and k1 must be coprimes

    If k1 is equal to 0:
    >>> decodeCeasarModified("whatever", 4, 0)
    Traceback (most recent call last):
        ...
    ValueError: k1 has to be greater than 0

    Word has to only consist of letters from english alphabet:
    >>> encodeCeasarModified('CRYP09RAPHY', 1, 0)
    Traceback (most recent call last):
        ...
    ValueError: word can't contain a number
    """
    if(isinstance(word, int) or any(chr.isdigit() for chr in word)):
      raise ValueError("word can't contain a number")

    word = str(word).replace(" ", "").upper()

    n=26

    if(k1 <= 0):
      raise ValueError("k1 has to be greater than 0")

    if(math.gcd(n, k1) != 1):
      raise ValueError("n and k1 must be coprimes")

    abc = string.ascii_uppercase
    output = ""
    for letter in word:
      idx = abc.index(letter)
      outidx = ((idx+(n-k0)))*k1**(phi(n)-1) % n
      output += abc[outidx]

    return output

# If every test is good it wont print anything
doctest.run_docstring_examples(encodeCeasarModified, globals(), name="Modified Ceasar Cypher Encoder")
doctest.run_docstring_examples(decodeCeasarModified, globals(), name="Modified Ceasar Cypher Decoder")  

## Opis algorytmu
### Szyfrowanie
Szyfrowanie odbywa się przez wczytanie indeksu litery z alfabetu i potraktowania go wzorem matematycznym, który nam dostarczono. Dokładniej liczba jest zwielokrotniona i przesunięta, a modulo 26 mówi nam jaką literą z alfabetu będzie teraz. Z uzyskanego w taki sposób indeksu korzystamy wybierając odpowiednią literę z alfabetu w budowaniu zakodowanego słowa.

### Deszyfrowanie
Deszyfracja odbywa się przez wczytanie indeksu litery z alfabetu i potraktowanie go wzorem matematycznym, który nam dostarczono. Dokładniej liczba jest przesuwa spowrotem o przesunięcie z szyfrowania i mnożona przez liczbę odwrotą modulo do k1 z szyfrowania [można taką liczbę uzyskać na kilka sposobów, jednym z nich jest funckja totient Eulera, zaimplementowałem ją w naiwny sposób, przechodząc przez każdą wartość od <0, n) i zliczając każdą liczbę która jest względnie pierwsza do n.] modulo 26 mówi nam jaką literę powinniśmy wybrać z alfabetu aby zbudować odkodowane słowo.

# Zadanie 5
## Treść
Zaimplementuj kryptosystem bazujący na tablicy Vigenere’a (1 punkt)
## Rozwiązanie

In [ ]:
def encodeVigenere(word, password):
    """Return encoded word in Vigenere algorithm according to password provided. 

    Base case #1:
    >>> encodeVigenere('CRYPTOGRAPHY', 'BREAK')
    'DICPDPXVAZIP'

    Base case #2:
    >>> encodeVigenere('CRYPTOGRAPHY', 'BATMOBILE')
    'DRRBHPOCEQHR'

    Password is empty:
    >>> encodeVigenere('CRYPTOGRAPHY', '')
    Traceback (most recent call last):
        ...
    ValueError: password can't be empty

    Password is a number:
    >>> encodeVigenere('CRYPTOGRAPHY', 1)
    Traceback (most recent call last):
        ...
    ValueError: password can't contain a number

    Word is a number:
    >>> encodeVigenere(1, "BREAK")
    Traceback (most recent call last):
        ...
    ValueError: word can't contain a number
    """
    if(isinstance(word, int) or any(chr.isdigit() for chr in word)):
      raise ValueError("word can't contain a number")
    if(isinstance(password, int) or any(chr.isdigit() for chr in password)):
      raise ValueError("password can't contain a number")
    if(password == ""):
      raise ValueError("password can't be empty")

    word = str(word).replace(" ", "").upper()
    password = str(password).replace(" ", "").upper()

    n=26

    while(len(password) < len(word)):
      password += password

    abc = string.ascii_uppercase
    output = ""
    for wletter, pletter in zip(word, password):
      widx = abc.index(wletter)
      pidx = abc.index(pletter)

      idx = (widx + pidx) % n
      output += abc[idx]

    return output

def decodeVigenere(word, password):
    """Return decoded word in Vigenere algorithm according to password provided. 

    Base case #1:
    >>> decodeVigenere('DICPDPXVAZIP', 'BREAK')
    'CRYPTOGRAPHY'

    Base case #2:
    >>> decodeVigenere('DRRBHPOCEQHR', 'BATMOBILE')
    'CRYPTOGRAPHY'

    Password is empty:
    >>> decodeVigenere('CRYPTOGRAPHY', '')
    Traceback (most recent call last):
        ...
    ValueError: password can't be empty

    Password is a number:
    >>> decodeVigenere('CRYPTOGRAPHY', 1)
    Traceback (most recent call last):
        ...
    ValueError: password can't contain a number

    Word is a number:
    >>> encodeVigenere(1, "BREAK")
    Traceback (most recent call last):
        ...
    ValueError: word can't contain a number
    """
    if(isinstance(word, int) or any(chr.isdigit() for chr in word)):
      raise ValueError("word can't contain a number")
    if(isinstance(password, int) or any(chr.isdigit() for chr in password)):
      raise ValueError("password can't contain a number")
    if(password == ""):
      raise ValueError("password can't be empty")

    word = str(word).replace(" ", "").upper()
    password = str(password).replace(" ", "").upper()

    n=26

    while(len(password) < len(word)):
      password += password

    abc = string.ascii_uppercase
    output = ""
    for wletter, pletter in zip(word, password):
      widx = abc.index(wletter)
      pidx = abc.index(pletter)

      idx = (widx - pidx) % n
      output += abc[idx]

    return output

# If every test is good it wont print anything
doctest.run_docstring_examples(encodeVigenere, globals(), name="Vigenere Encoder")
doctest.run_docstring_examples(decodeVigenere, globals(), name="Vigenere Decoder")  

## Opis algorytmu
### Szyfrowanie
Na wstępie dodaję hasło same do siebie, aby miało długość równą lub większą od słowa kodowanego.
Następnie przechodzę jednoczenie przez hasło i słowo wejściowe i dla każdej pary liter dodaje ich indeksy modulo 26, tak uzyskany indeks jest wykorzystany aby wczytać odpowiednia literę z alfabetu do wyjścia.

### Deszyfrowanie
Na wstępie dodaję hasło same do siebie, aby miało długość równą lub większą od słowa kodowanego.
Następnie przechodzę jednoczenie przez hasło i słowo wejściowe i dla każdej pary liter odejmuję ich indeksy modulo 26, tak uzyskany indeks jest wykorzystany aby wczytać odpowiednia literę z alfabetu do wyjścia.

# Sprawdzenie działania podczas zajęć 
Porównując do wyników z http://testowaniealgorytmow2020.site

In [ ]:
print(encodeRailFence('ARTURPOROWSKIMARZEC', 3))
print(encodeMatrixCypherA('ARTURPOROWSKIMARZEC'))
print(encodeMatrixCypherB('ARTURPOROWSKIMARZEC', 'ZONVENIENCE'))
print(encodeMatrixCypherC('ARTURPOROWSKIMARZEC', 'ZONVENIENCE'))
print(encodeCeasarModified('ARTURPOROWSKIMARZEC', 5, 7))
print(encodeVigenere('ARTURPOROWSKIMARZEC', 'ZREAK'))

AROIZRUPRWKMRETOSAC
TUARRROPWOIMSAKECRZ
WRRRCSOETMPZORIUAAK
WRAROTIEPORKZUMCASR
FUIPUGZUZDBXJLFUYHT
ZIXUBOFVOGRBMMKQQIC


In [ ]:
print(decodeRailFence('AROIZRUPRWKMRETOSAC', 3))
print(decodeMatrixCypherA('TUARRROPWOIMSAKECRZ'))
print(decodeMatrixCypherB('WRRRCSOETMPZORIUAAK', 'ZONVENIENCE'))
print(decodeMatrixCypherC('WRAROTIEPORKZUMCASR', 'ZONVENIENCE'))
print(decodeCeasarModified('FUIPUGZUZDBXJLFUYHT', 5, 7))
print(decodeVigenere('ZIXUBOFVOGRBMMKQQIC', 'ZREAK'))

ARTURPOROWSKIMARZEC
ARTURPOROWSKIMARZEC
ARTURPOROWSKIMARZEC
ARTURPOROWSKIMARZEC
ARTURPOROWSKIMARZEC
ARTURPOROWSKIMARZEC
